In [1]:
import pandas as pd
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_432dd80c81fe44369ef5acdda0543239.csv"
df=pd.read_csv(url,header=None,na_values=["?"])
df.head()

,0,1,2,3,4,5,6,7,8
0,mid,low,excellent,mid,stable,stable,stable,15.0,A
1,mid,high,excellent,high,stable,stable,stable,10.0,S
2,high,low,excellent,high,stable,stable,mod-stable,10.0,A
3,mid,low,good,high,stable,unstable,mod-stable,15.0,A
4,mid,mid,excellent,high,stable,stable,stable,10.0,A


In [2]:
df.mode()

,0,1,2,3,4,5,6,7,8
0,mid,mid,good,mid,stable,stable,stable,10.0,A
1,NaN,NaN,NaN,NaN,unstable,NaN,NaN,NaN,NaN


In [3]:
df=df.fillna(10)

In [4]:
df=df.rename(columns={8:"Class"})
y=df[["Class"]]

In [5]:
y=y.replace(['A '],['A'])

In [6]:
y.groupby("Class")["Class"].count()

Class
A    64
I     2
S    24
Name: Class, dtype: int64

In [7]:
X_sub=df.iloc[:,0:7]

In [8]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X_sub=X_sub.apply(le.fit_transform)

In [9]:
level=pd.DataFrame(df.iloc[:,7])

In [10]:
X=pd.concat([X_sub,level],axis=1)

In [11]:
df=pd.concat([X,y],axis=1)

In [13]:
dfA=df[df["Class"]=="A"]
dfS=df[df["Class"]=="S"]
dfI=df[df["Class"]=="I"]

In [14]:
X_A=dfA.drop(["Class"],axis=1)
y_A=dfA[["Class"]]

X_S=dfS.drop(["Class"],axis=1)
X_S=X_S+1
y_S=dfS[["Class"]]

In [15]:
X=pd.concat([X_A,X_S],axis=0)
y=pd.concat([y_A,y_S],axis=0)

In [16]:
import numpy as np
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

# feature extraction. We are going to output the selection scores for all features and select the features with the highest scores.
test = SelectKBest(score_func=f_classif, k="all")
fit = test.fit(X, np.asarray(y).reshape(y.shape[0],))
features = fit.transform(X)
# summarize scores
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
5,273.636364
2,74.176298
4,68.231405
1,37.978936
6,37.915528
3,31.853710
0,31.272727
7,3.145213


In [18]:
X_scores=scores[scores["scores"]>30]
len(X_scores)

7

In [19]:
Xp=X.iloc[:,X_scores.index]
Xp.head()

,5,2,4,1,6,3,0
0,1,0,0,1,1,2,2
2,1,0,0,1,0,0,0
3,2,1,0,1,0,0,2
4,1,0,0,2,1,0,2
10,1,1,0,2,1,2,2


In [20]:
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

In [44]:
from sklearn.ensemble import RandomForestClassifier
Forest=RandomForestClassifier(random_state=0)
scores=cross_val_score(Forest,Xp,np.asarray(y).reshape(y.shape[0],),cv=10)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  1.0 +/- 0.0


In [28]:
from sklearn.ensemble import GradientBoostingClassifier
Gradient=GradientBoostingClassifier(max_features="sqrt",random_state=0)
scores=cross_val_score(Gradient,Xp,np.asarray(y).reshape(y.shape[0],),cv=10)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  1.0 +/- 0.0


In [43]:
from sklearn import svm
SVM=svm.SVC(kernel="linear",random_state=0)
scores=cross_val_score(SVM,Xp,np.asarray(y).reshape(y.shape[0],),cv=10)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  1.0 +/- 0.0
